In [1]:
import fasttext

In [2]:
!python3 preparation.py

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/fedorkurusin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/fedorkurusin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

shape: (50425, 2)

percrentiles 2%: 5.0, 97%: 368.0

100%|████████████████████████████████████| 47260/47260 [01:16<00:00, 615.76it/s]

End of processing shape: (26137, 2)

write files for fasttext training in data/train.txt and data/test.txt



In [3]:
model1 = fasttext.train_supervised('data/train.txt')

def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

print_results(*model1.test('data/test.txt'))

sample_size=6809
precision=0.51
recall=0.51


Read 0M words
Number of words:  49054
Number of labels: 4
Progress: 100.0% words/sec/thread: 3245196 lr:  0.000000 avg.loss:  0.993234 ETA:   0h 0m 0s


In [4]:
model2 = fasttext.train_supervised('data/train.txt', epoch=25)

print_results(*model2.test('data/test.txt'))

Read 0M words
Number of words:  49054
Number of labels: 4
Progress:  96.4% words/sec/thread: 5169037 lr:  0.003581 avg.loss:  0.586790 ETA:   0h 0m 0s

sample_size=6809
precision=0.93
recall=0.93


Progress: 100.0% words/sec/thread: 4012654 lr:  0.000000 avg.loss:  0.575986 ETA:   0h 0m 0s


In [5]:
model3 = fasttext.train_supervised('data/train.txt', epoch=10, lr=1.0)

print_results(*model3.test('data/test.txt'))

Read 0M words
Number of words:  49054
Number of labels: 4
Progress:  88.0% words/sec/thread: 5621662 lr:  0.119709 avg.loss:  0.529208 ETA:   0h 0m 0s

sample_size=6809
precision=0.95
recall=0.95


Progress: 100.0% words/sec/thread: 3247825 lr:  0.000000 avg.loss:  0.493704 ETA:   0h 0m 0s


In [6]:
model4 = fasttext.train_supervised('data/train.txt', epoch=10, lr=1.0, wordNgrams=2)

print_results(*model4.test('data/test.txt'))

Read 0M words
Number of words:  49054
Number of labels: 4
Progress:  87.4% words/sec/thread: 2828153 lr:  0.126280 avg.loss:  0.334200 ETA:   0h 0m 0s

sample_size=6809
precision=0.99
recall=0.99


Progress: 100.0% words/sec/thread: 2148141 lr:  0.000000 avg.loss:  0.297725 ETA:   0h 0m 0s


In [9]:
model5 = fasttext.train_supervised('data/train.txt', autotuneValidationFile='data/test.txt')

print_results(*model5.test('data/test.txt'))

Progress: 100.0% Trials:  205 Best score:  1.000000 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  49054
Number of labels: 4
Progress:  82.9% words/sec/thread: 6560391 lr:  0.101797 avg.loss:  0.066278 ETA:   0h 0m 0s

sample_size=6809
precision=1.0
recall=1.0


Progress: 100.0% words/sec/thread: 6343108 lr:  0.000000 avg.loss:  0.055880 ETA:   0h 0m 0s


In [10]:
# model6 = fasttext.train_supervised('data/train.txt', autotuneValidationFile='data/test.txt', autotuneMetric="f1:__label__household")

# print_results(*model6.test('data/test.txt'))

In [16]:
print_results(*model5.test('data/test.txt', threshold=0.99))

sample_size=6809
precision=1.0
recall=0.97


In [17]:
model5.save_model('optimized.model')

### f1 weighted and f1 macro

In [35]:
from sklearn.metrics import f1_score

In [43]:
texts = []
true = []
pred = []
with open("data/test.txt", "r") as fo:
    for line in fo.readlines():
        l = line.strip()
        txt = " ".join(l.split()[1:])
        texts.append(txt)
        true.append(l.split()[0].replace("__label__", ""))
        pred.append(model5.predict(txt)[0][0].replace("__label__", ""))

In [46]:
f1_macro = f1_score(true, pred, average='macro')
f1_weighted = f1_score(true, pred, average='weighted')

print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")

F1 Score (Macro): 0.535
F1 Score (Weighted): 0.697


In [10]:
# model5.quantize(input='data/train.txt', retrain=True)

# model5 = fasttext.load_model("optimized.model")

# # Пример классификации одного заголовка
# model5.predict("brand new xiaomi tv")